You find yourself staring at a dataset with tens or hundreds of thousands of rows. Maybe you want to get up-to-date FOIA contact details for all government departments in your country, or to find out which political donors have links to the fossil fuels industry. What do you do?

This kind of work is time-consuming and challenging. However, Large Language Models (LLMs) like those powering ChatGPT can help journalists automate simple research and classification tasks that would take an unreasonably long time to do otherwise.

In this session, we'll outline how to use LLMs, search engines and web scraping to help us identify links between Donald Trump and his donors. You can [download the notebook](https://github.com/nicucalcea/ddj-wiki/blob/main/ai/python-classification-agent.ipynb) and run it yourself, or you can [run it in the cloud using Google Colab](https://colab.research.google.com/github/nicucalcea/ddj-wiki/blob/main/ai/python-classification-agent.ipynb). Both links also in the sidebar to the right, or at the bottom of the page on mobile.

## Install and load libraries

First, we'll need to install some libraries to help us call different LLMs and retrieve search results.

In [ ]:
%pip install pandas
%pip install pydantic-ai
%pip install duckduckgo-search
%pip install trafilatura

Then, we'll import those libraries.

In [ ]:
import os

# maths
import pandas as pd

# scraping
from urllib import request
import ssl
import requests
from trafilatura import extract

# validation
from typing import List, Tuple
from pydantic import BaseModel, Field
from typing import Literal
import json

# AI
from pydantic_ai import Agent, Tool
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool

# make async work in jupyter
import nest_asyncio
nest_asyncio.apply()

Finally, we need to define some of the API keys we'll be using. You can get your own [OpenAI API key](https://platform.openai.com/account/api-keys) or from another provider.

<span hidden="">Get a <a href="https://static.globalwitness.org/dataharvest-api-keys.txt" target="_blank">temporary API key here</a>.</span>

In [ ]:
#| eval: false
os.environ["OPENAI_API_KEY"] = ""
os.environ["BRIGHTDATA_API_KEY"] = ""
os.environ["BRIGHTDATA_ZONE"] = ""

## Prep data

We'll work with some data from the US Federal Election Commission (FEC) on [donations to Donald Trump's inaugural committee](https://docquery.fec.gov/cgi-bin/forms/C00894162/1889684/f132). Because a downloadable version isn't provided, we'll scrape the data directly from the website.

In [3]:
response = request.urlopen("https://docquery.fec.gov/cgi-bin/forms/C00894162/1889684/f132",
                           context=ssl._create_unverified_context())
html = response.read()

fec_raw = pd.read_html(html)[0]

We need to fix some of the formatting issues, like removing dollar signs and converting dates to a more standard format.

In [4]:
fec = fec_raw.copy()
fec = fec.iloc[:-1] # the last row is a summary

fec['Date Donation Received'] = pd.to_datetime(
    fec['Date Donation Received'], 
    format='%m/%d/%Y'  # Format for "31/12/2025"
)

fec['Donation Amount'] = fec['Donation Amount'].str.replace('$', '', regex=False)
fec['Donation Amount'] = pd.to_numeric(fec['Donation Amount'])

fec = fec.drop(columns=["Donor's Aggregate Donations To Date"])

Some donors donated multiple times, so we'll want to group them together and only classify them once.

In [ ]:
fec_total = fec.groupby('Name')['Donation Amount'].sum().reset_index()
fec_total = fec_total.sort_values(by='Donation Amount', ascending=False)
fec_total = fec_total.head(5) # To keep things simple, we're only going to use the first 5 rows of the dataset.
os.makedirs('data', exist_ok=True)
fec_total.to_csv('data/fec_unclassified.csv', index=False)

We should be ready to go. Let's see what the data looks like.

In [7]:
fec_total

,Name,Donation Amount
618,PILGRIM'S PRIDE CORPORATION,5000000.00
652,"RIPPLE LABS, INC.",4889345.33
824,WARREN A STEPHENS,4000000.00
139,CHEVRON PRODUCTS COMPANY,2000000.00
667,"ROBINHOOD MARKETS, INC.",2000000.00


## Grounding

Large Language Models are prone to [hallucinations](https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence)). If you ask an LLM a question it doesn't know the answer to, [it will confidently make up a plausible-sounding answer](https://ddj.nicu.md/ai/) that is completely wrong. This is particularly the case with lesser known organisations or individuals that wouldn't feature promionently in the training data.

One way to minimise (but not completely eliminate) hallucinations is by "grounding" the LLM with some factual data like documents or databases. This is sometimes a technique called Retrieval-Augmented Generation (RAG).

In our case, we want to allow out agent to search the internet to look for additional information about each donor.

We could try scraping search results from Google like we did with the FEC data, but Google doesn't like that, and they will put various roadblocks in your way such as CAPTCHAs, rate limits or even blocking your IP address. Instead, they want you to use their [Custom Search API](https://developers.google.com/custom-search/v1/overview), which is paid and limited to 10k queries per day.

Other search engines like Brave provide generous free tiers and more reasonable pricing, and there are some services that specifically cater to AI applications, like [Tavily](https://tavily.com/) and [Perplexity](https://docs.perplexity.ai/home).

We'll use the DuckDuckGo API since it's free (but rate-limited), doesn't require signing up for an API key, and comes included with the agents framework we'll be using.

We also want to give our AI the ability to visit each of those search results and extract the text from there. We'll use a library called [Trafilatura](https://github.com/adbar/trafilatura) since it extract the main text without headers, footers and other irrelevant bits that we probably don't need for the classification.

In [8]:
def extract_text(urls: List[str]) -> List[Tuple[str, str]]:
    """
    Extract text content from a list of URLs.
    
    Args:
        urls: A list of URLs to scrape
        
    Returns:
        A list of tuples containing (url, extracted_text)
    """
    results = []

    for url in urls:
        # print(f"Scraping {url}...")
        try:
            response = requests.get(url, timeout=30, verify=False)
            response.raise_for_status()
            extracted_text = extract(response.text, output_format="markdown")
            results.append((url, extracted_text))
        except requests.RequestException as e:
            # print(f"Error scraping {url}: {str(e)}")
            results.append((url, f"Error: {str(e)}"))

    return results

We'll eventually give our agent access to both the duckduckgo search function, as well as the `extract_text` function that we just created.

## Structured output

Most modern LLMs are trained to produce structured output, which means they can return data in a specific format. This is useful for producing consistent outputs when we're asking the model to classify or research multiple items.

We can define the output format we expect using a [Pydantic model](https://docs.pydantic.dev/latest/concepts/models/).

In [9]:
class TrumpConnection(BaseModel):
    donor_industry: str
    type_of_connection: Literal['Family', 'Business', 'Political', 'Other', 'No connection', 'Don\'t know or not enough information']
    explanation: str
    source_urls: str

## Agents

If we have an idea about what we want to search for, we could use a simple workflow where we first search for the donor's name, then extract the text from the first 5-10 results, and finally feed them to an LLM. An earlier version of this project used this approach, and it worked well enough.

Since then, AI agents have become a lot more prominent. Agents are systems in which LLMs are allowed to choose their own steps, use tools appropriate for different tasks and make decisions about when to stop.

The code required to create an agent from scratch is fairly simple, but there are several frameworks which come with batteries included. We'll use [PydanticAI](https://ai.pydantic.dev), but most frameworks provide a similar set of features.

First, let's create the agent. We'll choose a model to use, give it a system prompt, define the tools it should have access to, and the structured output we expect.

In [ ]:
agent = Agent(
    'openai:gpt-4.1-mini',
    system_prompt='Answer the question to the best of your abilities, using the tools at your disposal (`duckduckgo_search_tool` and `extract_text`).',
    tools=[duckduckgo_search_tool(), Tool(extract_text)],
    output_type=TrumpConnection,
    model_settings={'temperature': 0.0}
)

Let's check whether that works. We'll ask the agent to classify a single donor, and see what it returns.

In [11]:
result = agent.run_sync(
    'What is the connection between WARREN A STEPHENS and Donald Trump?'
)

In [12]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Answer the question to the best of your abilities, using the tools at your disposal (`duckduckgo_search_tool` and `extract_text`).', timestamp=datetime.datetime(2025, 5, 20, 9, 31, 23, 606273, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='What is the connection between WARREN A STEPHENS and Donald Trump?', timestamp=datetime.datetime(2025, 5, 20, 9, 31, 23, 606277, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions=None, kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='duckduckgo_search', args='{"query":"Warren A Stephens Donald Trump connection"}', tool_call_id='call_irBHmSWOHaLPRjFJFPPfpt2k', part_kind='tool-call')], usage=Usage(requests=1, request_tokens=226, response_tokens=23, total_tokens=249, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0}), model_name='gpt-

As you can see above, our agent started by searching the internet, read the search results, selected a few of the most relevant ones, read the text from those pages, and finally produced an output in the format we defined earlier.

## Running on the full dataset

We now have a working agent that can search the internet, read websites and return structured outputs. We can now have it run on each row of our dataset and return a full classification for each donor.

First, let's define a simple function that incorporates the agent run shown above.

In [14]:
def classify_donor(donor: str):
    result = agent.run_sync(f'What is the connection between {donor} and Donald Trump?')
    return result.output

Now, we can run it on the scraped data from the FEC website.

In [15]:
def apply_classify_donor(df):
    # Apply the classify_donor function to each row
    df['classification'] = df.apply(lambda row: classify_donor(donor=row['Name']), axis=1)
    
    # Convert the Pydantic objects to dictionaries
    classification_dicts = df['classification'].apply(lambda x: x.model_dump())
    
    # Create a new DataFrame from the classification dictionaries
    # This ensures the index matches the original dataframe
    classification_df = pd.DataFrame(classification_dicts.tolist(), index=df.index)
    
    # Combine original data with classification data
    result_df = pd.concat([df.drop(['classification'], axis=1), classification_df], axis=1)
    
    return result_df

In [16]:
fec_classified = apply_classify_donor(fec_total)

In [17]:
fec_classified

,Name,Donation Amount,donor_industry,type_of_connection,explanation,source_urls
618,PILGRIM'S PRIDE CORPORATION,5000000.00,Meat processing / Poultry production,Political,"Pilgrim's Pride Corporation, a major poultry p...",https://www.notus.org/donald-trump/inauguratio...
652,"RIPPLE LABS, INC.",4889345.33,Cryptocurrency / Blockchain technology,Political,"Ripple Labs, Inc., a blockchain and cryptocurr...",https://abcnews.go.com/US/sec-drops-case-crypt...
824,WARREN A STEPHENS,4000000.00,Investment Banking and Financial Services,Political,Warren A. Stephens is an American businessman ...,"https://en.wikipedia.org/wiki/Warren_Stephens,..."
139,CHEVRON PRODUCTS COMPANY,2000000.00,Oil and energy,Political,The connection between Chevron Products Compan...,https://www.bbc.com/news/articles/c62zzv02r3vo
667,"ROBINHOOD MARKETS, INC.",2000000.00,Financial Technology (FinTech),Political,"Robinhood Markets, Inc. made a $2 million dona...",https://www.sahmcapital.com/news/content/robin...


We now have a full classification for each donor which we can export to a CSV file, analyse further or review manually.

## Next steps

The example above shows a minimal working example of how to use LLMs to research a dataset. However, there are several ways in which this process could be improved. Here are some ideas:

- **[Play with the prompt](https://github.com/Global-Witness/augmenta/)**. We asked a simple question, but the more specific you are, the better the results. Write a good prompt, include examples and iterate.
- **[Make the AI reason](https://github.com/Global-Witness/augmenta/blob/main/docs/tools.md#:~:text=MCP%20servers%20are%3A-,sequential%20thinking,--%20Allows%20the%20agent)**. Allowing an LLM to "reason" about its next steps can help it produce better outputs.
- **[Use different search engines](https://github.com/Global-Witness/augmenta/blob/main/docs/search.md)**. DuckDuckGo is free and good for a prototype. However, their API isn't meant to be used this way and will often return errors, especially when used on bigger datasets. It also doesn't return the best results. I recomment switching to Google, potentially via a SERP API.
- **Try other models**. I like to use gpt-4.1-mini because it is reasonably fast, cheap and "smart" enough for most tasks. If you find it often gets it wrong, you can use the smarter gpt-4.1 or a "reasoning" model like o3.
- **Validate the output**. While we did code the classifier to return structured output, you can go beyond that and validate the output (ie. check that the sources are valid URLs, check that the explanation includes exact quotes from the sources, etc).
- **[Save your progress](https://github.com/Global-Witness/augmenta/blob/main/docs/cache.md)**. Things can and will go wrong! The search engine may fail, the LLM may be down (Claude often is), your Python script may throw an unexpected error. If you don't want to lose progress, you should save your progress to file or a database so you can pick up from where you left off.
- **Async/Multithreading**. In my tests, each classification takes about 20-50 seconds, but it can take longer if the LLM is trying to accesss an unresponsive website. If you have a lot of rows to classify, you should consider running multiple agents in parallel.
- **Verify**. LLMs are still dumb and shouldn't be trusted! Manually verify the classifications if you're going to publish the results.

# A better way

I've used the approach described above for several projects at Global Witness, including our [annual classification of fossil fuel donors at COP](https://globalwitness.org/en/campaigns/fossil-fuels/how-we-used-ai-to-help-us-find-lobbyists-at-cop/).

As the codebase grew to account for the suggestions described above, I decided to package everything into a Python library called **[Augmenta](https://github.com/Global-Witness/augmenta/)**. You can use Augmenta to build agents without writing any code. It has a few features that make life easier, such as:

- Built-in search and text extraction tools
- Support for multiple LLMs
- Asynchronous processing
- Automatic caching to save progress
- Support for [third-party tools via MCP](https://github.com/Global-Witness/augmenta/blob/main/docs/tools.md)
- Logfire integration for monitoring and debugging

Let's see how it works.

Because Augmenta is an interactive command-line tool, it doesn't work great in a Jupyter notebook. Instead, we'll run it in a separate terminal locally.

First, we need to install Augmenta.

In [ ]:
%pip install augmenta

Next, we need to create a configuration file. This is a YAML file that contains all the settings for the agent, including the search engine to use, the LLM to use and the prompts. It may look something like this.

In [ ]:
yaml_config = """
input_csv: data/fec_unclassified.csv
output_csv: data/fec_classified.csv
model:
  provider: openai
  name: gpt-4.1-mini
search:
  engine: brightdata_google
prompt:
  system: You are an expert researcher whose job is to research the connections between political donors and Donald Trump.
  user: |
    # Instructions

    Your job is to research {{Name}}, a donor to Donald Trump's inaugural fund. Your will determine how Donald Trump and {{Name}} are connected, whether via family, business ties, political ties or something else. Don't consider donations to the inaugural fund as a connection, we're only interested in connections before or after the donation.

    ## Searching guidelines

    In most cases, you should start by searching for "{{Name}}" and "Donald Trump". Where relevant, remove redundant words like "INC", "LLC", "GROUP", etc from the search query. If you need to perform another search, try to refine it by adding relevant keywords. Note that each case will be different, so be flexible and adaptable. Unless necessary, limit your research to two or three searches.

    With each search, select a few sources that are most likely to provide relevant information. Access them using the tools provided. Be critical and use common sense. ALWAYS cite your sources.

    Now, please proceed with your analysis and classification of {{Name}}'s connections to Donald Trump.
structure:
  donor_industry:
    type: str
    description: What industry does this company, individual or organisation belong to?
  type_of_connection:
    type: str
    description: How are they connected to Donald Trump?
    options:
      - Family
      - Business
      - Political
      - Other
      - No connection
      - Don't know or not enough information
  explanation:
    type: str
    description: A few paragraphs explaining your decision in English, formatted in Markdown. In the explanation, link to the most relevant sources from the provided documents. Include at least one inline URL.
# mcpServers:
#   - name: sequential-thinking
#     command: npx
#     args:
#       - "-y"
#       - "@modelcontextprotocol/server-sequential-thinking"
examples:
  - input: "Melinda Hildebrand"
    output:
      donor_industry: Energy
      type_of_connection: Political
      explanation: |
        President Donald Trump nominated River Oaks Donuts owner and energy executive Melinda Hildebrand [to serve as U.S. Ambassador to Costa Rica](https://www.congress.gov/nomination/119th-congress/55/21). In May 2024, [the Financial Times reported](https://www.ft.com/content/c89bbfc4-80db-4f3f-8d63-3aeb46ae91a9) the Hildebrand and her husband hosted a campaign event for then-presidential candidate Trump, who was seeking donations from power players in the U.S. energy industry. [The Daily Beast reported](https://www.thedailybeast.com/trump-goes-small-at-fundraisers-as-harris-goes-big-all-over/) that the Hildebrands also served on the host committee of a Trump fundraising dinner in Colorado. According to the Daily Beast, the host committee gave or raised $500,000 per couple.
logfire: true
"""

# Write the string directly to the file
with open("data/augmenta.yaml", 'w') as file:
    file.write(yaml_config)

Now, we just to run Augmenta.

Because it's a CLI tool, we need to run it in a terminal. If you're running this notebook locally, it's best to run it in your preferred terminal instad of the notebook. If you're running it in Google Colab, you can start a terminal by running the following command in a code cell.

In [ ]:
#| eval: false
!bash

Next, you want to run the following command in the terminal.

```bash
augmenta data/augmenta.yaml -v
```

Note that you'll need the API keys for OpenAI and BrightData in your environment (we did that at the top of the notebook).

This approach is faster (parallel processing), more reliable (caching), more transparent (logging via logfire) and easier to use (no need to write code). It also allows you to use the same configuration file for different projects, so you can reuse your work.

You can also [extend Augmenta with custom tools](https://github.com/Global-Witness/augmenta/blob/main/docs/tools.md) via MCP servers. That means you can make it reason before each step, give it access to databases and APIs, and even allow it to write its own Python code (useful for processing data, doing maths, etc.).

Feel free to check the [README](https://github.com/Global-Witness/augmenta) and get in touch if you have any questions or suggestions.